In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pandas as pd
import pickle

In [16]:
model = load_model('model.h5')

with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender = pickle.load(file)

with open('onehot_encoder_geo.pkl','rb') as file:
    onehot_encoder_geo = pickle.load(file)

with open('scaler.pkl','rb') as file:
    scaler = pickle.load(file)

In [3]:
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [15]:
df = pd.DataFrame(input_data,index=[0])
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [17]:
label_encoder_gender.classes_

array(['Female', 'Male'], dtype=object)

In [18]:
df['Gender'] = label_encoder_gender.transform(df['Gender'])
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [19]:
test_encoder = onehot_encoder_geo.transform(df[['Geography']])
test_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 1 stored elements and shape (1, 3)>

In [21]:
test_encoder_df = pd.DataFrame(test_encoder.toarray(),columns = onehot_encoder_geo.get_feature_names_out(['Geography']))
test_encoder_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [22]:
df = pd.concat([df.drop('Geography',axis=1),test_encoder_df],axis=1)
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [24]:
df = scaler.transform(df)
df

c:\Users\vaibh\Python\ANN Classification\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


array([[-6.7681861 ,  0.73902774, -3.68814088, -1.97778513, -1.21847468,
        -1.24715903, -0.11888692,  0.92443458, -1.74618097,  1.00450338,
        -1.91524949, -1.90861117]])

In [25]:
prediction = model.predict(df)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step


In [26]:
print(prediction)

[[0.0003392]]


In [27]:
probability = prediction[0][0]

if probability>0.5:
    print("Yes")
else:
    print("No")

No
